In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu
from DataLoader import DataLoader
import matplotlib.pyplot as plt

import numpy as np

In [2]:
training_generator = DataLoader(64, "train")
val_generator = DataLoader(64, "val")

EOFError: Ran out of input

In [ ]:
lossHistory = []
valHistory = []
class NnomCallbacks(tf.keras.callbacks.Callback):
  def on_batch_end(self, batch, logs=None):
    lossHistory.append(logs['loss'])
    if batch and batch % 1000 == 0:
      pred = self.model.predict(val_generator)
      loss = (pred.flatten() - val_generator.labels.flatten()[0:len(val_generator)*64])**2
      loss = np.sum(loss) / len(loss)
      valHistory.append(loss)

      plt.plot(valHistory)
      plt.savefig("val.png", dpi=200)
      plt.clf()

      plt.plot(lossHistory[20:])
      plt.savefig("loss.png", dpi=200)
      plt.clf()
      
      print(" val_loss : {}".format(loss))

      self.model.save("models/model-{}-{}".format(len(valHistory), int(loss*10000)))

In [ ]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(98304,)))
model.add(Dense(384, activation='sigmoid'))
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model = tf.keras.models.load_model("production/")

In [ ]:
model.fit(training_generator, validation_data=val_generator, epochs=1, callbacks=[NnomCallbacks()])

In [ ]:
import time
model.save("models/model-{}".format(int(time.time())))